<a href="https://colab.research.google.com/github/m-bashari-m/vehicle-color-recognition/blob/main/src/5_rgb_xyz_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py

--2022-06-08 20:57:36--  https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11788 (12K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]  11.51K  --.-KB/s    in 0s      

2022-06-08 20:57:36 (122 MB/s) - ‘utils.py’ saved [11788/11788]



In [2]:
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [3]:
import pandas as pd
import numpy as np
import os

import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub

!pip install -q tensorflow-io
import tensorflow_io as tfio

from utils import ModelCreator, get_train_val_ds, get_class_weight

     |████████████████████████████████| 25.9 MB 1.2 MB/s 


In [4]:
BATCH_SIZE = 32
IMG_SIZE = (256, 256)
AUTOTUNE = tf.data.AUTOTUNE
N_CLASSES = 16
HUB_URL = 'https://tfhub.dev/google/bit/m-r50x1/1'

In [5]:
dataset_dir = os.path.join('drive', 'MyDrive', 'cars')
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')

In [6]:
train_ds, _ = get_train_val_ds(train_dir, val_dir, batch_size=BATCH_SIZE, img_size=IMG_SIZE)

Found 16580 files belonging to 16 classes.
Found 3508 files belonging to 16 classes.


In [7]:
classes, class_weight = get_class_weight()

In [8]:
def combine_rgb_xyz(img_batch):
  img_batch_xyz = tfio.experimental.color.rgb_to_xyz(img_batch)
  return (img_batch + img_batch_xyz) / 2

In [9]:
train_ds = (
    train_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    map(lambda img, lbl: (combine_rgb_xyz(img), lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

In [14]:
rgb_xyz_creator = ModelCreator(HUB_URL, 'RGB_XYZ-model')
rgb_xyz_model = rgb_xyz_creator.make_model(img_size=IMG_SIZE)

Model: "RGB_XYZ-model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 2048)              23500352  
                                                                 
 dense_2 (Dense)             (None, 600)               1229400   
                                                                 
 dropout_1 (Dropout)         (None, 600)               0         
                                                                 
 dense_3 (Dense)             (None, 16)                9616      
                                                                 
Total params: 24,739,368
Trainable params: 1,239,016
Non-trainable params: 23,500,352
_________________________________________________________________


In [15]:
callbacks = rgb_xyz_creator.get_callbacks()

In [16]:
history = rgb_xyz_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=25,
                        class_weight=class_weight)

Epoch 1/25
519/519 [==============================] - 191s 344ms/step - loss: 1158.2117 - auc: 0.4725 - accuracy: 0.6395
Epoch 2/25
519/519 [==============================] - 175s 334ms/step - loss: 1703.9325 - auc: 0.5370 - accuracy: 0.6982
Epoch 3/25
519/519 [==============================] - 173s 331ms/step - loss: 1564.8438 - auc: 0.5940 - accuracy: 0.7411
Epoch 4/25
519/519 [==============================] - 174s 333ms/step - loss: 1205.7815 - auc: 0.6347 - accuracy: 0.7706
Epoch 5/25
519/519 [==============================] - 176s 336ms/step - loss: 935.1781 - auc: 0.6603 - accuracy: 0.7874
Epoch 6/25
519/519 [==============================] - 178s 341ms/step - loss: 805.5470 - auc: 0.6739 - accuracy: 0.7978
Epoch 7/25
519/519 [==============================] - 174s 333ms/step - loss: 717.5233 - auc: 0.6864 - accuracy: 0.8056
Epoch 8/25
519/519 [==============================] - 174s 333ms/step - loss: 614.1190 - auc: 0.7106 - accuracy: 0.8218
Epoch 9/25
519/519 [================

In [17]:
rgb_xyz_model.save('./drive/MyDrive/checkpoints/rgb-xyz-model-25.h5')

# ***Second*** day

In [ ]:
rgb_xyz_model = tf.keras.models.load_model('./drive/MyDrive/checkpoints/rgb-xyz-model-25.h5',
                                           custom_objects={'KerasLayer': hub.KerasLayer})

In [ ]:
history =rgb_xyz_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=40,
                        initial_epoch=25,
                        class_weight=class_weight)

In [ ]:
rgb_xyz_model.save('drive/MyDrive/checkpoints/rgb-xyz-model-40.h5')

In [ ]:
history =rgb_xyz_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=50,
                        initial_epoch=40,
                        class_weight=class_weight)

Epoch 41/50
533/533 [==============================] - 196s 365ms/step - loss: 13.2826 - auc: 0.9172 - accuracy: 0.9344
Epoch 42/50
533/533 [==============================] - 184s 343ms/step - loss: 12.0273 - auc: 0.9200 - accuracy: 0.9369
Epoch 43/50
533/533 [==============================] - 195s 364ms/step - loss: 11.6070 - auc: 0.9223 - accuracy: 0.9363
Epoch 44/50
533/533 [==============================] - 197s 366ms/step - loss: 9.9959 - auc: 0.9295 - accuracy: 0.9412
Epoch 45/50
533/533 [==============================] - 198s 368ms/step - loss: 10.7260 - auc: 0.9251 - accuracy: 0.9387
Epoch 46/50
533/533 [==============================] - 202s 376ms/step - loss: 9.4814 - auc: 0.9301 - accuracy: 0.9424
Epoch 47/50
533/533 [==============================] - 194s 360ms/step - loss: 9.0810 - auc: 0.9326 - accuracy: 0.9441
Epoch 48/50
533/533 [==============================] - 194s 362ms/step - loss: 8.3280 - auc: 0.9345 - accuracy: 0.9447
Epoch 49/50
533/533 [=======================

In [ ]:
rgb_xyz_model.save('drive/MyDrive/checkpoints/rgb-xyz-model-50.h5')

In [ ]:
history =rgb_xyz_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=60,
                        initial_epoch=50,
                        class_weight=class_weight)

Epoch 51/60
533/533 [==============================] - 192s 356ms/step - loss: 8.1753 - auc: 0.9343 - accuracy: 0.9447
Epoch 52/60
533/533 [==============================] - 197s 368ms/step - loss: 7.7764 - auc: 0.9379 - accuracy: 0.9454
Epoch 53/60
533/533 [==============================] - 209s 389ms/step - loss: 8.0568 - auc: 0.9398 - accuracy: 0.9468
Epoch 54/60
533/533 [==============================] - 193s 359ms/step - loss: 7.6623 - auc: 0.9397 - accuracy: 0.9437
Epoch 55/60
533/533 [==============================] - 190s 354ms/step - loss: 7.6085 - auc: 0.9405 - accuracy: 0.9464
Epoch 56/60
533/533 [==============================] - 201s 375ms/step - loss: 7.2338 - auc: 0.9419 - accuracy: 0.9458
Epoch 57/60
533/533 [==============================] - 211s 393ms/step - loss: 6.7196 - auc: 0.9442 - accuracy: 0.9478
Epoch 58/60
533/533 [==============================] - 217s 404ms/step - loss: 6.9903 - auc: 0.9403 - accuracy: 0.9473
Epoch 59/60
533/533 [===========================

In [ ]:
rgb_xyz_model.save('drive/MyDrive/checkpoints/rgb-xyz-model-60.h5')

In [ ]:
history =rgb_xyz_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=65,
                        initial_epoch=60,
                        class_weight=class_weight)

Epoch 61/65
533/533 [==============================] - 194s 361ms/step - loss: 6.4579 - auc: 0.9457 - accuracy: 0.9491
Epoch 62/65
533/533 [==============================] - 191s 356ms/step - loss: 6.5886 - auc: 0.9428 - accuracy: 0.9468
Epoch 63/65
533/533 [==============================] - 191s 355ms/step - loss: 6.7383 - auc: 0.9431 - accuracy: 0.9479
Epoch 64/65
533/533 [==============================] - 187s 348ms/step - loss: 6.4305 - auc: 0.9449 - accuracy: 0.9484
Epoch 64: early stopping


In [ ]:
rgb_xyz_model.save_weights('drive/MyDrive/checkpoints/rgb-xyz-model-61.h5')